In [1]:
!pip install simpy

In [2]:
import simpy as sp
import random as rd

# Set constants
checkers = 35  # number of id checkers
scanners = 35  # number of scanners
random_seed = 500

# Set rates
arri_rate = 50  # passengers' arrival per minute
check_rate = 0.75  # passengers' boarding pass check per minute
min_scan = 0.5  # minimum scan time uniform distribution
max_scan = 1.0  # maximum scan time uniform distribution

# Set timing
run_time = 600  # minutes per simulation
reps = 100  # number of simulation replications

# Set global variables
avg_check_time = []  # average boarding check time per replication
avg_scan_time = []  # average scan time per replication
avg_wait_time = []  # average wait time in total per replication
avg_sys_time = []  # average system time in total per replication


# Build model
class System(object):
    def __init__(self, env):
        self.env = env
        self.checker = sp.Resource(env, checkers)
        self.scanner = [sp.Resource(env, 1) for _ in range(scanners)]

    def check(self, psg):
        """Define a boarding pass check time"""
        yield self.env.timeout(rd.expovariate(1.0 / check_rate))

    def scan(self, psg):
        """Define scan time"""
        yield self.env.timeout(rd.uniform(min_scan, max_scan))


def passenger(env, name, s):
    global tot_through
    time_arri = env.now  # get arrival time of passenger

    with s.checker.request() as request:  # boarding pass check queue
        yield request
        t_start_check = env.now  # get when passenger starts being checked
        yield env.process(s.check(name))
        t_end_check = env.now  # get when passenger ends being checked

    with s.scanner[rd.randint(0, scanners - 1)].request() as request:
        yield request
        t_start_scan = env.now  # get when passenger starts being scanned
        yield env.process(s.scan(name))
        t_end_scan = env.now  # get when passenger ends being scanned

    time_done = env.now  # time passenger finishes
    sys_time.append(time_done - time_arri)
    tot_through += 1


def setup(env):
    i = 0
    s = System(env)
    while True:
        yield env.timeout(rd.expovariate(arri_rate))
        i += 1
        env.process(passenger(env, 'Passenger %d' % i, s))  # i-th passenger


# Run model
for i in range(reps):
    rd.seed(i)
    env = sp.Environment()
    tot_through = 0
    check_time = []
    scan_time = []
    sys_time = []

    # Run the simulation
    env.process(setup(env))
    env.run(until=run_time)

    # Calculate average times for this replication
    avg_sys_time.append(sum(sys_time[1:]) / tot_through)
    avg_check_time.append(sum(check_time[1:]) / tot_through)
    avg_scan_time.append(sum(scan_time[1:]) / tot_through)
    avg_wait_time.append(avg_sys_time[i] - avg_check_time[i] - avg_scan_time[i])

    print('%d : Replication %d times %.2f %.2f %.2f %.2f' % (tot_through,
                                                            i + 1,
                                                            avg_sys_time[i],
                                                            avg_check_time[i],
                                                            avg_scan_time[i],
                                                            avg_wait_time[i]))

# Print overall averages
print('--------Overall Average times for all passengers--------')
print('Average system time = %.2f' % (sum(avg_sys_time) / reps))
print('Average check time = %.2f' % (sum(avg_check_time) / reps))
print('Average scan time = %.2f' % (sum(avg_scan_time) / reps))
print('Average wait time = %.2f' % (sum(avg_wait_time) / reps))

27034 : Replication 1 times 37.04 0.00 0.00 37.04
27066 : Replication 2 times 30.41 0.00 0.00 30.41
27186 : Replication 3 times 32.02 0.00 0.00 32.02
27242 : Replication 4 times 29.88 0.00 0.00 29.88
27178 : Replication 5 times 30.86 0.00 0.00 30.86
27197 : Replication 6 times 32.55 0.00 0.00 32.55
26985 : Replication 7 times 33.42 0.00 0.00 33.42
27256 : Replication 8 times 32.79 0.00 0.00 32.79
27262 : Replication 9 times 31.37 0.00 0.00 31.37
27135 : Replication 10 times 33.56 0.00 0.00 33.56
27206 : Replication 11 times 33.34 0.00 0.00 33.34
27153 : Replication 12 times 33.80 0.00 0.00 33.80
27302 : Replication 13 times 28.31 0.00 0.00 28.31
27268 : Replication 14 times 29.05 0.00 0.00 29.05
27201 : Replication 15 times 31.65 0.00 0.00 31.65
27018 : Replication 16 times 31.35 0.00 0.00 31.35
27228 : Replication 17 times 32.05 0.00 0.00 32.05
27119 : Replication 18 times 31.88 0.00 0.00 31.88
27137 : Replication 19 times 30.14 0.00 0.00 30.14
27238 : Replication 20 times 31.22 0.00 